In [1]:
import requests as req   # pip install requests
from requests import get
from bs4 import BeautifulSoup as bf #pip install BeautifulSoup
import openpyxl #pip install openpyxl
# from openpyxl.drwing.image import Image 
import pandas as pd #pip install pandas
import os
from datetime import datetime, date
import datetime 
import time
import re
from IPython.display import Image
import json
import io
import jpype
import asposecells
from urllib.parse import urlparse, urljoin

In [2]:
URLs = [ 
        'https://www.ettoday.net/news/focus/%E6%94%BF%E6%B2%BB/',  #政治
        'https://www.ettoday.net/news/focus/%E7%A4%BE%E6%9C%83/',   #社會
        'https://www.ettoday.net/news/focus/%E5%A4%A7%E9%99%B8/',   #大陸   時間會有問題
        'https://www.ettoday.net/news/focus/%E7%94%9F%E6%B4%BB/',  #生活新聞
       ]# 示範幾個主要的category

df_rows = []

In [3]:
for URL in URLs:
    request = req.get(URL)
    sp = bf(request.text, 'html.parser')
    sel = sp.find_all('div', {'class': 'piece clearfix'})[:20]
#     category = sp.find('em', class_='icon').find_next_sibling('span').text
    try:
        category = sp.find('em', class_='icon').find_next_sibling('span').text
    except AttributeError:
        category = sp.find('h2', class_='block_title').text.strip()




    for i in sel:
        link = i.find('a').get('href')    # 各個的連結   
        if not urlparse(link).scheme:
            link = urljoin(URL, link)
        
        Link_Content = req.get(link)
        sp_inner = bf(Link_Content.text, 'html.parser')
        
#         index = sp_inner.find('strong')
        header = sp_inner.find('h1', class_='title')
        publish_time = sp_inner.find('time').text
        Inner_contents = [p.text for p in sp_inner.find_all('p')[3:] if not p.find('strong')]
        end_index = next((i for i, p in enumerate(Inner_contents) if 
                          "分享給朋友：" in p), None)
       
        if header:  
            header = header.get_text(strip=True)  # 看標標題有沒有
        else:
            print("No header found on page:", link)
            
        if end_index is not None:
            Inner_contents = Inner_contents[:end_index]     

#         Contents = []   # 看有沒有其他方法
        
        print(category)
        print(header)
        print(link)
        print(publish_time)
        
        for Inner_content in Inner_contents:  
            print(Inner_content)
#             Contents.append(Inner_content)
        df_rows.append({
                    'Category': category,  
                    'Title': header,
                    'Link': link,
                    'Publish time': publish_time,
#                     'Content': Contents
                     'Content': '\n'.join(Inner_contents)
                    }
                        )

政治
快訊／卓榮泰：肅貪是檢視政府唯一標準　「絕不留情面跟模糊空間」
https://www.ettoday.net/news/20240709/2774020.htm

                                                            2024年07月09日 17:43                                                        
行政院長卓榮泰今（9日）天赴立法院進行施政總質詢，結束後，卓榮泰也發表感言，他強調，從今天開始，全國同步掃黑、肅貪行動已經展開，這是人民檢視政府唯一標準，過程中不容許有任何突發事情、希望全面展開，絕對不留情面也沒有上限，沒有人可以僥倖得逞、也沒有任何模糊空間。
卓榮泰表示，大院對行政院提出施政方針報告的質詢，到今天順利完成，感謝委員先進提出很多寶貴意見跟指教，這都是未來貴我兩院共同遵循、努力方向，創造對人民最大福祉，深深表示感謝，備詢中若干不及答覆部份，也會請各部會首長提出詳實的書面報告。
卓榮泰指出，未來還有更多工作要共同努力，總質詢結束後，行政院會以雙主軸開啟國家重大建設，第一經濟發展、第二強化資安，這個月份就會召開首次經濟發展委員會，創新經濟、均衡台灣、包容成長三大主題全面推動。
卓榮泰強調，強化資安過程中，會從打詐、掃黑、肅貪、反毒強力執行，打詐過程在過去一段時間看到很多相關部會效果，希望發揮更強大力量、讓人民心安。至於掃黑、肅貪、反毒過程中，6月25日已經展開第一波靖平專案，內容針對綠能、肅貪、掃黑做全面性檢查跟防堵。
卓榮泰說明，從今天開始，全國同步掃黑、肅貪行動已經展開，這是人民檢視政府唯一標準，過程中不容許有任何突發事情、希望全面展開，絕對不留情面也沒有上限，沒有人可以僥倖得逞、也沒有任何模糊空間。
卓榮泰指出，希望貴我兩院在國家安全、社會安定、人民安心工作上繼續，下個會期，大院裡面還有對於行政院未來國家總預算以及若干重大法案，拜託一併支持，還有人事同意權部份，期待適時時間內通過。
政治
藍曾破門佔議場！　吳沛憶教韓國瑜何謂嚴重：集體破壞公物還賴帳43萬
https://www.ettoday.net/news/20240709/2773671.htm

                                

政治
桃園地院懷疑鄭文燦影響力？　黃國昌：看完裁定理由「差點沒昏倒」
https://www.ettoday.net/news/20240709/2773973.htm

                                                            2024年07月09日 16:25                                                        
前桃園市長鄭文燦捲入土地開發弊案，9日2度獲交保，桃園地院再裁定以1200萬元交保，即加保700萬元，檢方不服當庭再提出抗告；而鄭文燦的律師也於裁定結果出爐後，20分鐘內籌出700萬元現金辦保。民眾黨立委黃國昌直言「差點沒昏倒」，桃園地院竟然會懷疑鄭文燦影響力，如此率爾認定有收錢的鄭文燦仍否認犯行，但沒勾串滅證之虞。
針對鄭文燦涉貪二度交保，20分鐘籌到700萬交保金，黃國昌9日下午於臉書發文表示，「20分鐘再送到700萬元，收錢的鄭文燦二度交保，協商結束後，看到桃園地院重新裁定鄭文燦貪污1200萬交保的新聞稿，差點沒昏倒。」
黃國昌提及，在裁定理由中，桃園地院認為「被告卸任市長一職後迄今已逾1年半，是否仍於桃園市政府有影響力仍有疑義，且被告所犯涉及對價關係之構成要件，檢察官亦未釋明有哪些公務員知悉此等事實。」先不論桃園地院是不是「忘了」鄭文燦2個月前都還是行政院副院長，長年在民進黨內權傾天下，竟然還會懷疑鄭文燦的影響力？竟然如此率爾認定雖然有收錢的鄭文燦仍否認犯行，但並沒有勾串滅證之虞？
黃國昌指出，行賄的人直接拿錢到市長官邸、收賄金流都已經被掌握，送錢的人也已羈押禁見，然後法院考量「被告之政商地位，認為本案准予新台幣1200萬元交保得以替代羈押」，如此悖離常理的裁定，如何找回人民對司法的信心？果不其然，實力雄厚的鄭文燦，短短20分鐘內就送到700萬元現金，已經閃人了。
政治
檢方聲押鄭文燦押不成！　王鴻薇：賴系、英系鬥到司法跟檢調了
https://www.ettoday.net/news/20240709/2773970.htm

                                                            2024年07月09日 16:14                       

政治
北市中山商圈恐團滅？　顏若芳喊話蔣萬安：師大商圈殷鑑不遠
https://www.ettoday.net/news/20240709/2773831.htm

                                                            2024年07月09日 13:49                                                        
台北市「南西中山商圈」大量商家近來收到台北市政府都發局通知違反《台北市土地使用分區管理自治條例》，並限期2個月改善，引發數十間商家連署呼籲搶救商圈。北市議員顏若芳表示，南西心中山已是北市指標商圈之一，若北市府執意將其消滅，就會重蹈師大商圈覆轍，受害的都是青年創業家以及在地產業。
「心中山線型公園」自2019年底建設完畢開幕後，因其現代化的設計、便捷的交通，成為台北市城市美學的優秀案例之一，更在2023年被英國文化刊物《Time Out》選為「全球最酷的 40 個街區」之一。再加上既有的百貨商場，躍升成台北市假日休閒的新去處。
時至今日，心中山更變成許多文創市集、青年創業家、工作室的聚集地，每逢假日就有相當多年輕人來逛逛，南西商圈對青年回流中山區、活化在地產業有相當大的助益，更是台北觀光、文創的重鎮之一。
顏若芳說，2015年的「師大商圈之亂」，當時師大商圈也吸引了許多青年創業家和工作室，但後來因市府無能調解商家與住民的衝突，反而讓大量的商家撤出，讓師大商圈一蹶不振，時至今日才在想辦法行銷、補破網。
顏若芳指出，當今的台北市長蔣萬安，過去就是中山區立委，如今竟執意要打擊中山區的產業，消滅南西中山商圈，令人瞠目結舌，「難道大市長就那麽瞧不起小市民嗎」？
顏若芳呼籲，北市府不應將師大商圈的慘痛教訓複製到南西中山商圈，正視問題，提出社區住戶與店家共存的方案，應優先展延限改期限。她認為，都發局應與地方住戶、商家共同討論現行《土管條例》在住三、四用地開業的附帶條件是否不符合時宜，因若據此徹查北市全境，恐有大量商家都將被消滅。
社會
「遊艇大亨」咖啡廳隨機下藥！她喝1口秒頭暈　急躲美髮店逃劫
https://www.ettoday.net/news/20240709/2774041.htm

                               

社會
台中姨騎樓騎車「嗆行人不讓」：我沒錯！再罵警察欺負老百姓
https://www.ettoday.net/news/20240709/2773997.htm

                                                            2024年07月09日 16:55                                                        

第二警分局永興所昨日14時37分接獲民眾報案，稱北區崇德路一段騎樓有車禍發生。經員警到場了解，是一名高姓女騎士於騎樓騎車，鳴按喇叭要求行人羅女讓道，機車車籃碰觸到羅女，雙方發生口角。

警方到場了解情況時，高婦情緒激動地說：「我車子要過來，他們就一直走、不讓我，我又沒怎樣！我要趕時間啊，我住在這邊20多年，我車子要停這裡，沒什麼事你叫警察來，我沒有犯錯，他們故意找我麻煩。」員警則說：「沒什麼事不用這麼大聲，你先下來。」反而讓高婦暴怒，稱警察只會欺負老百姓。
高婦一頓咆哮怒罵後，便不顧員警阻攔快閃離去，羅女因為沒有受傷，也不願提告追究。不過，警方表示，高婦已經違反《道交條例》第45條，可處可處600元以上1800元以下罰鍰；不服稽查取締部分則依第60條最高處3萬元罰鍰、吊扣駕照1年；未禮讓行人一事，將調閱監視器查察，若有可依第44條開罰最高6000元罰鍰。
第二分局提醒民眾，遵守交通法規不僅是法律責任，更是對他人安全的基本尊重，針對「車輛不禮讓行人」之駕駛行為，內政部警政署已列為重點取締項目，呼籲駕駛人行車時應讓行人先行，共同維護行人路權。
社會
法務部檢察首長交接典禮　北檢「3位檢察長」同框成焦點
https://www.ettoday.net/news/20240709/2773984.htm

                                                            2024年07月09日 16:42                                                        
法務部9日辦理3位檢察首長就職宣誓典禮，高雄高分檢檢察長費玲玲屆期卸任，遺缺由台南高分檢檢察長朱家崎接任，行政執行署署長黃玉垣則獲新派擔任台南高分檢檢察長，此外，新任的台北地檢署檢

大陸
妹子用「頭砸西瓜」腦震盪住院　她傻眼：未料到西瓜沒熟
https://www.ettoday.net/news/20240709/2773945.htm

                                                            2024年07月09日 16:36                                                        
山東煙台一名女子近日與友人買了兩顆西瓜，準備享受夏日的消暑聖品。女子見友人輕鬆用手掰開一顆西瓜，便突發奇想用頭部砸另一顆西瓜，怎料，西瓜不但沒被砸開，她反而還輕微腦震盪住院了。
據《中華網新聞》報導，該名女子表示，友人向她炫耀一項特殊技能「不用刀就可以開西瓜」，「我看我朋友用手就很輕鬆彈開了西瓜，所以才想說用更誇張的方式開西瓜。」


女子直呼，「但是我沒料到另一顆西瓜沒有熟啊！不但計劃沒成功，還因為腦震盪住院了。」友人們得知消息後，還紛紛帶著西瓜去醫院探病。該事件引起熱議，有人表示，「醫護人員可能會一直去病房看她，因為想看看被西瓜『擊敗』的人長怎樣。」

一名專家對此表示，該行為有高度風險，西瓜外皮或許看起來殺傷力不大，但其實整顆西瓜是非常堅硬的，用頭部去撞擊是非常容易造成腦震盪的，「頭部作為人體關鍵且最脆弱的部份，任何情況下都應該避免直接碰撞硬物。」
大陸
化工油、食用油混裝！陸「中字輩國企」產品全被下架
https://www.ettoday.net/news/20240709/2773927.htm

                                                            2024年07月09日 15:54                                                        
中國大陸傳出有油罐車載完媒油之後，儲存槽未經清洗，就直接又裝運食用大豆油，甚至連大陸國企「中儲糧」都涉入其中。消息經過幾日的發酵後，中儲糧旗下的食用油品牌「金鼎」已在多個電商平台被下架。
根據《澎湃新聞》報導，7月9日中午，在京東APP上，以金鼎、金鼎食用油等為關鍵字搜尋，已無法搜索到金鼎官方旗艦店或自營旗艦店資訊，也沒有其他店舖在售訊息。
在淘寶APP上，以金鼎、金鼎食用油

大陸
中國IPO撤單潮下　8000多名證券業從業人員離職
https://www.ettoday.net/news/20240709/2773710.htm

                                                            2024年07月09日 11:45                                                        
中國加嚴資本市場監管，掀起IPO撤單潮，影響了證券業從業狀況，統計顯示，今年截至目前為止，有8000多名從業人員離開證券業；IPO業務愈趨於集中在大型發行機構。
第一財經報導表示，據Wind數據不完全統計，今年截至目前為止，僅21家券商實現IPO承銷保薦業務收入；隨著擬上市的企業大量撤回，約52家保薦機構、600多家保薦代表人白忙了一場。
據Wind數據不完全統計，今年截至目前為止，僅46家公司首發上市，年減74%；首發募集資金總額人民幣338.356億元（約新台幣1514.27億元），年減84%。
目前累計318家擬IPO企業終止審核，包括主機板116家、創業板92家、科創板58家、北交所52家。這些擬IPO企業終止審核，涉及52家保薦機構、636位保薦代表人，其中以中信證券IPO終止審核企業數量最多，達42家。
第一財經報導指出，證券業與資本市場密切相關，屬於週期性特別強的行業，今年在資本市場下滑、IPO業務緊縮、券商業績承壓等狀況下，證券業從業人員降薪力道有增無減。
金融領域數據資料庫Choice統計顯示，今年已有8000多名從業者離開了證券業；截至今年7月8日，證券業從業人員總數為34.38萬人。
在擬IPO企業數量大幅減少背景下，IPO業務收入也愈往證券業大型、龍頭機構集中，能夠參與的券商數量正在減少中。
證券時報日前報導剖析，今年以來撤回IPO申請集中發生在4月下旬後，這與IPO門檻提高有關。今年4月，滬深北3大交易所分別發布系列新規，IPO門檻提高、財務不達標成為企業主動撤回IPO申請一大原因。
現場檢查也是企業撤回IPO申請原因之一。中國證監會在2021年1月底發布「首發企業現場檢查規定」，其彼時強調，將大幅提高現場檢查比率，在現場檢查中若發現欺詐發行、財務造假等線索，一經查實將加以嚴懲。
大陸
想把撫養權給前妻！父「每隔兩三天」

生活
暑假指定日外帶單球冰淇淋買一送一　双聖登台40周年祭優惠
https://travel.ettoday.net/article/2773961.htm

        2024年07月9日 18:10    
美式家庭餐廳SWENSEN’S 双聖歡慶登台40周年，在暑假推出限定活動，除了兒童餐平日半價優惠、外帶單球冰淇淋買一送一，8月31日前，當日雙北氣溫如果衝破40度，餐廳招待40組消費者免費吃單球冰淇淋。

今年為双聖登台40周年，品牌推出暑假限定活動，首波即日起至8月31日，平日期間大人點主餐，可享兒童餐半價優惠；7月13日至7月14日首度參加有趣市集，現場除了供應香草、草莓、啾啾巧克力、蘋果蔓越莓雪酪、夏威夷果、棉花糖堅果巧克力及摩卡杏仁等7種冰淇淋口味，以及限定款「凍華田脆酷力聖代」。

双聖夏日還有雙重冰淇淋優惠，7月22至7月28日、8月22日至8月28日外帶單球冰淇淋，可享有買一送一，眾多人氣冰淇淋口味任君挑選。而即日起至8月31日止，凡中央氣象局公布當日台北市、新北市最高氣溫破40度，双聖將招待40組消費者免費吃單球冰淇淋。

另外，有著「北海道霜淇淋之神」美名的人氣冰淇淋品牌「CREMIA」，推出夏季最新「北海道富良野哈密瓜霜淇淋」，每一口都能吃到哈密瓜香結合濃醇奶香，甜蜜滋味超濃郁，7月10日起於全台門市上市，每支售價150元，售完為止，推薦餅皮搭配覆盆子口味，也提供原味、巧克力選擇。
※本文版權所有，非經授權，不得轉載！[ETtoday著作權聲明]※



生活
疑建商搞鬼！中山店家前腳收公文後腳接電話：要不要都更
https://house.ettoday.net/news/2774065

                  2024-07-09 18:08                
新中山商圈恐團滅？北市中山商圈近年來發展成為40大特色商圈之一，不過近來卻傳出疑似遭到建商與投資客惡意大量檢舉，指出商家不符合土地使用分區條例，商家也陸陸續續收到都發局公文要求限期改善，但商家前腳才剛收到都發局的公文，後腳竟馬上接到建商電話詢問，是否有意願都更，讓店家質疑根本就是建商看上中山商圈的都更大肥肉。
據了解，中山商圈主要為住四用地，僅能有一般零售業進駐，但隨著商圈的發展也有越來越多的餐飲業與零售業陸續進駐。根據現有法規來看，餐飲業者所

生活
全家現做茶1分鐘製成！比手搖店快1倍　新品「好喝奶茶」直逼霸主
https://www.ettoday.net/news/20240709/2773901.htm

                                                            2024年07月09日 17:27                                                        
搶攻手搖飲商機！全家便利商店私品茶宣布推出2大系列，「搖搖茶吧」主打果茶、奶茶等潮流口味，「福爾摩沙茶館」提供現煮台灣茶。並以多元有料、天然少添加、24小時現做外送為3大特色，推出超過30款飲品，製成一杯茶平均約1分鐘，比坊間手搖飲速度快1倍，朝著「全國最大連鎖茶飲店」目標邁進。

台灣手搖飲一年賣出超過10億杯，全家瞄準千億市場，陸續投資2.5億佈建機台設備、產線專利、商品研發，全面搶攻茶金商機。根據調查，消費者喜愛的前三手搖飲類型為原茶、果茶、奶茶，也習慣在飲用前搖一搖。
全家私品茶「搖搖茶吧」系列在全台超過4200間密集據點，主打果茶、奶茶等流行口味，此波推出「寒天甘蔗檸檬」特大杯65元，7月10日起還有全新品「好喝奶茶」媲美手搖店霸主，特大杯55元，7月10日～23日會員嘗鮮價49元。
而「福爾摩沙茶館」系列，選用台灣首座通過國際雨林認證優質茶園的茶葉，近來推出「紅柚椰果白烏龍」及「紅柚椰果紅茶」，將新鮮果茶與現煮茶香融合，還有椰果帶來咀嚼感。預計7月底在全台有近500間全家門市提供「福爾摩沙茶館」現煮茗茶。

全家商品部本部長李慶賢提到，全家因為流程優化，製作現做茶一杯平均約1分鐘，比坊間手搖飲速度快1倍。董事長葉榮廷則說，擴大現做茶的規模，不是為了要打擊手搖飲店，事實上無論什麼產品，目的都不是跟外界搶，而是為了鞏固品牌，並提供消費者多元選擇。
為了提供更便利的體驗，全家外送自取平台FamiNow推出搖搖吧優惠活動，7月10日～7月23日使用FamiNow購買搖搖茶吧，可享全品項買2送1優惠（限同品項），自取訂單滿149元再送30元購物金。


另外，7-ELEVEN將於7月10日起推出一系列懷舊冰品、飲品，包括CITY CAFE首推「冰椪糖拿鐵」，以府城百年傳統古早味椪糖，結合CITY CAFE拿鐵，誕生出獨特香甜焦香、咖

In [4]:
df = pd.DataFrame(df_rows)


today = datetime.date.today().strftime('%Y-%m-%d')
file_name = 'ET Today' + today 
file_path = 'C:/Users/Administrator/INTERN/testing'

df.to_excel(os.path.join(file_path, file_name + '.xlsx'), 
            index=False, encoding='UTF-8-Sig', engine='openpyxl')

C:\Users\Administrator\anaconda3\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
